# YvesBlue Backend Challenge
## Carbon Analytic Calculation
### Goal: Utilizing this Jupyter Notebook (and additional libraries/files/tools as needed) implement a function that can be utilized to calculate the (strawman) Adjusted CO2 Total emissions for a company. 
### Notes: The calculation is laid out in this [image](calculation.png?raw=true). The data is stored inside of the [included data file](data.json?raw=true). Your code structure can take whatever you believe to be best (individual function, class, module, etc.). Please implement the code as well as write tests for the calculation, showcasing how you would write tests for such requirements. Provide instructions with your submission for how to call your implemented function as well as how to run the tests.
## Author: @github_username (Full Name)
## Date: YYYY-MM-DD

In [31]:
%load_ext autoreload
%autoreload 2

In [55]:
## Import Libraries
import decimal as d
import json
import os

import numpy
import pandas as pd

In [51]:
with open("data.json") as f:
    data = json.load(f, parse_float=d.Decimal)

In [52]:
data_df = pd.DataFrame(data).set_index("ISIN")

In [73]:
data_df.to_excel("data.xlsx")

In [44]:
data_df.loc[~data_df["CO2 Analytic"].isnull()]

,Total Energy Use,Total CO2 Equivalents Emissions,Renewable Energy Purchased,Renewable Energy Produced,Carbon Credit Value,CO2 Analytic
ISIN,,,,,,
US0000000006,445000000,64963.62994,9515.906056,24720.733080,10556.113590,53170.89797
US0000000007,518000000,89288.41927,32904.927480,4051.865565,8902.593156,80180.67966
US0000000011,810000000,219384.62410,18766.813620,13746.231470,47047.328130,171647.98800


In [74]:
data_df.columns

Index(['Total Energy Use', 'Total CO2 Equivalents Emissions',
       'Renewable Energy Purchased', 'Renewable Energy Produced',
       'Carbon Credit Value', 'CO2 Analytic'],
      dtype='object')

In [82]:
data_df["Renewable Energy Purchased"].values[0]

Decimal('10576.00479')

In [105]:
## Demonstrate Solution with Calculation Function
# class is in file adj_total_emissions.py
import adj_total_emissions as ate

In [106]:
computer = ate.AdjTotalComputer()
data_df.loc[:, "adj emissions"] = data_df.apply(computer.get_carbon_emissions, 1)
data_df.head()

,Total Energy Use,Total CO2 Equivalents Emissions,Renewable Energy Purchased,Renewable Energy Produced,Carbon Credit Value,CO2 Analytic,adj emissions
ISIN,,,,,,,
US0000000000,7000000,94972.49198,10576.00479,96652.16115,8171.323352,NaN,81902.98845798661959084857143
US0000000001,80000000,306900.6192,31961.79405,38372.1921,57387.1,NaN,247545.06647179149299525000
US0000000002,153000000,171320.1651,62884.91047,10689.83474,84303.7259,NaN,86464.06504146892092018300654
US0000000003,226000000,2250.272892,12949.74785,27468.66907,45713.96386,NaN,-44835.87919176843822473716814
US0000000004,299000000,132696.7376,42559.65263,69567.95366,17515.5,NaN,111694.6424697395493396521739


In [88]:
## Test Calculation Function
computer.get_carbon_emissions(data_df.loc["US0000000011"])

Decimal('171647.9879632202198968448148')

In [111]:
import unittest
w_answers = [r for r in data if r.get("CO2 Analytic")]

In [112]:
class TestAdjTotalComputer(unittest.TestCase):
    def setUp(self):
        self.computer = ate.AdjTotalComputer()
        
    def test_init(self):
        self.assertEqual(self.computer.psi_prod*20, 1)
        
    def test_answers(self):
        for rec in w_answers:
            self.assertTrue(
                abs(self.computer.get_carbon_emissions(rec)-rec["CO2 Analytic"]) < d.Decimal(0.0001))
            
    def test_min(self):
        rec = {'Total Energy Use': 10,
             'Total CO2 Equivalents Emissions': 10,
             'Renewable Energy Purchased': 18,
             'Renewable Energy Produced': 20,
             'Carbon Credit Value': 5}
        use_max_purch = self.computer.get_carbon_emissions(rec)
        
        rec['Renewable Energy Purchased'] = 14
        self.assertEqual(use_max_purch, 0)
        self.assertTrue(self.computer.get_carbon_emissions(rec)>use_max_purch)
        rec['Renewable Energy Purchased'] = 16
        self.assertEqual(self.computer.get_carbon_emissions(rec), 0)


In [113]:
# run this cell for unittests
unittest.main(argv=[''], verbosity=2, exit=False)

test_answers (__main__.TestAdjTotalComputer) ... ok
test_init (__main__.TestAdjTotalComputer) ... ok
test_min (__main__.TestAdjTotalComputer) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK
